# Authorship Identification

## Section 4.3: Method 3 - Basic Features & Word2Vec + FNN & RNN

### 1. Dataset & Data Loader

In [51]:
import numpy as np
import pandas as pd

data_tran = pd.read_json('../data/data2/data_tran.json', orient='records', lines=True)
data_test = pd.read_json('../data/data2/data_test.json', orient='records', lines=True)

n_tran = data_tran.shape[0]
n_test = data_test.shape[0]

x_tran_coauthors = np.load("../data/data2/x_tran_coauthors.npy")
x_tran_venue_a   = np.load("../data/data2/x_tran_venue_a.npy")
x_tran_venue_b   = np.load("../data/data2/x_tran_venue_b.npy")
x_tran_text_a    = np.load("../data/data2/x_tran_text_a.npy")
x_tran_text_b    = np.load("../data/data2/x_tran_text_b.npy")

x_test_coauthors = np.load("../data/data2/x_test_coauthors.npy")
x_test_venue_a   = np.load("../data/data2/x_test_venue_a.npy")
x_test_venue_b   = np.load("../data/data2/x_test_venue_b.npy")
x_test_text_a    = np.load("../data/data2/x_test_text_a.npy")
x_test_text_b    = np.load("../data/data2/x_test_text_b.npy")

x_tran_title_doc2vec = np.load('../data/data2/x_tran_title_doc2vec.npy')
x_test_title_doc2vec = np.load('../data/data2/x_test_title_doc2vec.npy')

x_tran_abstract_doc2vec = np.load('../data/data2/x_tran_abstract_doc2vec.npy')
x_test_abstract_doc2vec = np.load('../data/data2/x_test_abstract_doc2vec.npy')

y_tran_basic = np.load("../data/data2/y_tran.npy")
x_tran_basic = np.concatenate((x_tran_coauthors, x_tran_venue_a, x_tran_venue_b, x_tran_text_a, x_tran_text_b), axis=1)
x_test_basic = np.concatenate((x_test_coauthors, x_test_venue_a, x_test_venue_b, x_test_text_a, x_test_text_b), axis=1)

In [52]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

idxs = np.random.permutation(n_tran)
inxs_prop = int(0.85 * n_tran)
idxs_tran_indices = idxs[:inxs_prop]
idxs_vald_indices = idxs[inxs_prop:]

x_tran_a = torch.tensor(x_tran_basic[idxs_tran_indices], dtype=torch.float32).to(device)
x_vald_a = torch.tensor(x_tran_basic[idxs_vald_indices], dtype=torch.float32).to(device)
x_test_a = torch.tensor(x_test_basic, dtype=torch.float32).to(device)

y_tran   = torch.tensor(y_tran_basic[idxs_tran_indices], dtype=torch.float32).to(device)
y_vald   = torch.tensor(y_tran_basic[idxs_vald_indices], dtype=torch.float32).to(device)
y_test   = torch.zeros((x_test_a.shape[0], y_tran.shape[1]), dtype=torch.float32)

data_tran_new = data_tran.iloc[idxs_tran_indices].reset_index(drop=True)
data_vald_new = data_tran.iloc[idxs_vald_indices].reset_index(drop=True)
data_test_new = data_test

In [53]:
import json

with open("../data/data2/x_tran_title_word_vectors.json", "r") as f:
    word_vectors_dict_title = json.load(f)

with open("../data/data2/x_tran_abstract_word_vectors.json", "r") as f:
    word_vectors_dict_abstract = json.load(f)

word_vectors_dict_title    = {word: np.array(vector) for word, vector in word_vectors_dict_title.items()}
word_vectors_dict_abstract = {word: np.array(vector) for word, vector in word_vectors_dict_abstract.items()}

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class ModelDataset(Dataset):
    def __init__(self, x1, y, data, word_vectors_dict_title, word_vectors_dict_abstract):
        self.x1 = x1
        self.y = y
        self.data = data
        self.word_vectors_dict_title = word_vectors_dict_title
        self.word_vectors_dict_abstract = word_vectors_dict_abstract

        self.max_title_len = self.get_max_length(data['title_text'])
        self.max_abstract_len = self.get_max_length(data['abstract_text'])

    def __len__(self):
        return len(self.data)

    def get_max_length(self, texts):
        return max(len(text.split()) for text in texts)

    def text_to_vector(self, text, word_vectors_dict, max_len):
        vectors = [word_vectors_dict.get(word, np.zeros_like(next(iter(word_vectors_dict.values())))) for word in text.split()]
        vectors = vectors[:max_len] + [np.zeros_like(vectors[0])] * (max_len - len(vectors))
        return torch.tensor(vectors, dtype=torch.float32) 

    def __getitem__(self, idx):
        x1 = self.x1[idx].clone().detach().float()
        
        x2 = self.text_to_vector(self.data.iloc[idx]['title_text'], 
                                 self.word_vectors_dict_title, 
                                 max_len = 20).to(device)
        x3 = self.text_to_vector(self.data.iloc[idx]['abstract_text'], 
                                 self.word_vectors_dict_abstract, 
                                 max_len = 200).to(device)

        y = self.y[idx].clone().detach().float()

        return x1, x2, x3, y

In [55]:
dataset_tran = ModelDataset(x_tran_a, y_tran, data_tran_new, word_vectors_dict_title, word_vectors_dict_abstract)
dataset_vald = ModelDataset(x_vald_a, y_vald, data_vald_new, word_vectors_dict_title, word_vectors_dict_abstract)
dataset_test = ModelDataset(x_test_a, y_test, data_test_new, word_vectors_dict_title, word_vectors_dict_abstract)

datalod_tran = DataLoader(dataset_tran, batch_size=128)
datalod_vald = DataLoader(dataset_vald, batch_size=128)
datalod_test = DataLoader(dataset_test, batch_size=128)

### 2. Model Structure

In [56]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

class FNNRNN(nn.Module):
    def __init__(self, input_dim1, output_dim):
        super(FNNRNN, self).__init__()

        self.rnn_t = nn.RNN(input_size=100, hidden_size=50,  batch_first=True, nonlinearity='relu')
        self.rnn_a = nn.RNN(input_size=200, hidden_size=100, batch_first=True, nonlinearity='relu')

        self.fnn = nn.Sequential(
            nn.Linear(input_dim1+50+100, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x1, x2, x3):

        if len(x1.shape) == 1:
            x1 = x1.unsqueeze(0)
            x2 = x2.unsqueeze(0)
            x3 = x3.unsqueeze(0)
        
        x2_out, x2_hidden = self.rnn_t(x2) 
        x2_out_last = x2_out[:, -1, :]

        x3_out, x3_hidden = self.rnn_a(x3) 
        x3_out_last = x3_out[:, -1, :]

        x = torch.cat([x1, x2_out_last, x3_out_last], dim=1) 

        x = self.fnn(x)

        return x

### 3. Model Training

In [57]:
input_dim1 = x_tran_a.shape[1]
output_dim = y_tran.shape[1] 

model = FNNRNN(input_dim1, output_dim).to(device)
criterion = nn.BCELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.00001)

In [58]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(pred_label, true_label):
    pred_label = pred_label.int()
    true_label = true_label.int()
    pc = precision_score(true_label.cpu(), pred_label.cpu(), average='macro', zero_division=0)
    rc = recall_score(true_label.cpu(), pred_label.cpu(), average='macro', zero_division=0)
    f1 = f1_score(true_label.cpu(), pred_label.cpu(), average='macro', zero_division=0)
    return pc, rc, f1

In [59]:
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss):
        if self.best_loss is None or train_loss < self.best_loss - self.delta:
            self.best_loss = train_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

early_stopping = EarlyStopping(patience=10, delta=0.001)

In [ ]:
epochs = 500

for epoch in range(epochs):
    model.train()  
    total_loss = 0  

    for batch in datalod_tran:
        x1, x2, x3, y = [t.to(device) for t in batch]

        outputs = model(x1, x2, x3)
        loss = criterion(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(datalod_tran)

    if (epoch + 1) % 10 == 0:
        model.eval()
        y_tran_pred_prob, y_vald_pred_prob = [], []
        y_tran_true_labl, y_vald_true_labl = [], []

        with torch.no_grad():
            for batch in datalod_tran:
                x1_tran, x2_tran, x3_tran, y_tran = [t.to(device) for t in batch]
                y_tran_pred_prob.append(model(x1_tran, x2_tran, x3_tran).cpu())
                y_tran_true_labl.append(y_tran.cpu())

            y_tran_pred_prob = torch.cat(y_tran_pred_prob, dim=0)
            y_tran_pred_labl = (y_tran_pred_prob > 0.5).int()
            y_tran_true_labl = torch.cat(y_tran_true_labl, dim=0)

            for batch in datalod_vald:
                x1_vald, x2_vald, x3_vald, y_vald = [t.to(device) for t in batch]
                y_vald_pred_prob.append(model(x1_vald, x2_vald, x3_vald).cpu())
                y_vald_true_labl.append(y_vald.cpu())

            y_vald_pred_prob = torch.cat(y_vald_pred_prob, dim=0)
            y_vald_pred_labl = (y_vald_pred_prob > 0.5).int()
            y_vald_true_labl = torch.cat(y_vald_true_labl, dim=0)

        tran_pc, tran_rc, tran_f1 = calculate_metrics(y_tran_pred_labl, y_tran_true_labl)
        vald_pc, vald_rc, vald_f1 = calculate_metrics(y_vald_pred_labl, y_vald_true_labl)

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
        print(f"Train - Precision: {tran_pc:.4f}, Recall: {tran_rc:.4f}, F1 Score: {tran_f1:.4f}")
        print(f"Val   - Precision: {vald_pc:.4f}, Recall: {vald_rc:.4f}, F1 Score: {vald_f1:.4f}\n")

        early_stopping(avg_loss)
        if early_stopping.early_stop:
            print("Early Stop!")
            break

model.eval()
y_test_pred_prob = []

with torch.no_grad():
    for batch in datalod_test:
        x1_test, x2_test, x3_test, _ = [t.to(device) for t in batch]
        y_test_pred_prob.append(model(x1_test, x2_test, x3_test).cpu())

    y_test_pred_prob = torch.cat(y_test_pred_prob, dim=0)
    y_test_pred_labl = (y_test_pred_prob > 0.5).int()

print(f"Test Prediction Shape: {y_test_pred_labl.shape}")

### 4. Model Prediction

In [ ]:
def generate_output_csv(x_test_a, y_test_pred_labl):
    
    result = []
    
    for i, row in enumerate(y_test_pred_labl):
        if ((x_test_a[i, :100] < 1).all() or (x_test_a[i, 100:200] == 0).all() or (x_test_a[i, 200:300] == 0).all() or (x_test_a[i, 300:400] == 0).all() or (x_test_a[i, 400:500] == 0).all()):
            result.append("-1")
        elif row.sum() == 0 or row[100] == 1:
            result.append("-1")
        else:
            indices = [str(idx) for idx, val in enumerate(row) if val == 1]
            result.append(" ".join(indices))
    
    result_df = pd.DataFrame({"ID": range(len(result)), "Predict": result})
    
    return result_df

generate_output_csv(x_test_a, y_test_pred_labl).to_csv("../data/data3/result_method3.csv", index=False)